In [6]:
# from utils import *
import random
# import Graph as G
import numpy as np
import pandas as pd
import pickle
import networkx as nx
import os


In [173]:
class Vertex(object):
    def __init__(self, key):
        self.key = key # 4 bytes
        self.neighbors = set()
        self.value=[None]

    def __init__(self, key,value):
        self.key = key
        self.neighbors = set()
        self.value=value

    def add_neighbor(self, neighbor):
        self.neighbors.add(neighbor)

    def __str__(self):
        return '{} neighbors: {} \n values: {}'.format(
            self.key,
            [x for x in self.neighbors]
            ,self.value[:3]
        )
    
    # get neighbor vertices
    def get_neighbors(self):
        return self.neighbors

class Graph(object):
    def __init__(self):
        self.verticies = {}

    def add_vertex(self, vertex):
        self.verticies[vertex.key] = vertex

    def get_vertex(self, key):
        try:
            return self.verticies[key]
        except KeyError:
            return None

    def __contains__(self, key):
        return key in self.verticies

    def add_edge(self, v1_info, v2_info):
        v1_key,v1_value=v1_info
        v2_key,v2_value=v2_info
        if v1_key not in self.verticies:
            self.add_vertex(Vertex(v1_key,v1_value))
        if v2_key not in self.verticies:
            self.add_vertex(Vertex(v2_key,v2_value))
        self.verticies[v1_key].add_neighbor(v2_key)

    def get_vertices(self):
        return self.verticies.keys()

    def __iter__(self):
        return iter(self.verticies.values())

    def __str__(self):
        return 'Graph({})'.format(dict(self.verticies))

In [189]:
class Indexer():
    def __init__(self, file_path='dataset/DB10K.csv', R= 17, L= 15, alpha = 2, K= 5, new_db = True):
        self.R = R
        self.L = L
        self.alpha = alpha
        self.k = K
        self.datasetPath = file_path
        self.new_db = new_db
        self.DBGraph=self.Initialize_Random_Graph()
        self.medoid=self.get_medoid()
        self.offset = 0
    # TODO: make another constructor

    def load_binary_data(self, binary_file_path):
    # Load the data from the binary file
        with open(binary_file_path, 'rb') as f:
            data = pickle.load(f)
       
        self.offset = data.iloc[0][0]
        # print(data)
        return data

    # Get vertices from DB and insert them into the graph
    def Initialize_Random_Graph(self):
        DBGraph=Graph()
        DBdata = self.load_binary_data(self.datasetPath)
        for row in DBdata.itertuples(index=True, name='Pandas'):
            dataKey = int(row[1])
            # print(dataKey)
            dataValue= np.array(row[2:],dtype=float)
            dataValue /= np.linalg.norm(dataValue)
            # print(dataValue)
            DBGraph.add_vertex(Vertex(dataKey, dataValue))

        size= len(DBGraph.verticies)
        if(size==0 or size==1):
            return

        #Building Edges
        for vertex in DBGraph:
            # we want R neighbors
            # TODO: Make random numbers from a set of keys
            for i in range(self.R):
                neighbor= DBGraph.get_vertex(int(random.random()*size) + self.offset)
                while(neighbor==vertex):
                    neighbor= DBGraph.get_vertex(int(random.random()*size) + self.offset)
                DBGraph.add_edge((vertex.key,vertex.value),(neighbor.key,neighbor.value))
        return DBGraph


    # gets euclidean distance between 2 vectors
    def get_distance(self,v1,v2):
        dist = np.dot(v1,v2) #/ (np.linalg.norm(v1) * np.linalg.norm(v2))
        # print(v1.shape, v2.shape)
        return 1.0 - dist
        # return  np.linalg.norm(v1-v2)


    # # gets medoid of a graph
    # def get_medoid(self):
    #     min_distance=10000000000000000000
    #     medoid=None
    #     for vertex in self.DBGraph:
    #         current_total_distance=0
    #         for vertex2 in self.DBGraph:
    #             if(vertex==vertex2):
    #                 continue
    #             dist=self.get_distance(vertex.value,vertex2.value)
    #             current_total_distance+=dist

    #         if(current_total_distance<min_distance):
    #             min_distance=current_total_distance
    #             medoid=vertex
    #     print(medoid.key)
    #     return medoid,min_distance

    #doesn't change with the measure of distance
    def get_medoid(self):
        vX = list(self.DBGraph.get_vertices())
        Embeddings = [self.DBGraph.verticies[i].value for i in vX]
        vMean = np.mean(Embeddings, axis=0)                               # compute centroid
        # i = np.argmin([sum((x - vMean)**2) for x in Embeddings])          # pick a point closest to centroid
        minIndex = self.DBGraph.get_vertex(np.argmin([sum((x - vMean)**2) for x in Embeddings]))
        # print(i)
        return minIndex # pick a point closest to centroid



    #gets arg min distance from any vertex in Anyset to Query and min dist
    def get_min_dist (self,AnyKeysSet,Query):
        min_dist=100000000
        min_vertex=None
        for vertexKey in AnyKeysSet:
            # print("vertex",vertex)
            # print("Query",Query[:3])
            vertex=self.DBGraph.get_vertex(vertexKey)
            dist=self.get_distance(vertex.value,Query)
            print(dist)
            if(dist<min_dist):
                min_dist=dist
                min_vertex=vertex
        return min_vertex,min_dist

    def get_min_dist_Key (self,AnyKeysSet,Query):
        # arrKey = list(AnyKeysSet)
        # arrEmb = np.array([self.DBGraph.get_vertex(i).value for i in arrKey])

        # a_norm = np.linalg.norm(arrEmb, axis=1)
        # b_norm = np.linalg.norm(Query)
        # dist = (arrEmb @ Query) / (a_norm * b_norm)

        # # minDist = np.linalg.norm(arrEmb, axis=1)
        # minIndex = np.argmin(dist)
        # return arrKey[minIndex], -1

        min_dist=50
        min_vertex=None
        for vertexKey in AnyKeysSet:
            # print("vertex",vertex)
            # print("Query",Query[:3])
            vertex=self.DBGraph.get_vertex(vertexKey)
            dist=self.get_distance(vertex.value,Query)
            # print(dist)
            if(dist<min_dist):
                min_dist=dist
                min_vertex=vertex.key
        return min_vertex,min_dist


    #initially, start is the medoid
    # s is a vertex, Query is a vector
    # k is a number, L is a number
    # TODO: change them to be indices instead of vertices to save ram
    def Greedy_Search(self,start,Query, k):
        search_List={start.key}
        Visited=set()
        #TODO: make the visited and the possible frontier set of indices instead of vertices to save ram.
        possible_frontier=search_List
        Query /= np.linalg.norm(Query)
        while possible_frontier != set():
            # print('possible_frontier',possible_frontier)
            p_star,_= self.get_min_dist_Key(possible_frontier,Query)

            # print('pstar',p_star)
            # if p_star==None:
            #     # break
            #     print('frontier: ')
            #     for v in possible_frontier:
            #         print(v)
            #     print(possible_frontier==set())
            search_List=search_List.union(self.DBGraph.get_vertex(p_star).neighbors)
            Visited.add(p_star)
            if(len(search_List)>self.L):
                #update search list to retain closes L points to x_q
                search_ListL_L=list(search_List)
                search_ListL_L.sort(key=lambda x: self.get_distance(self.DBGraph.get_vertex(x).value,Query))
                # only maintain L closest points
                search_ListL_L=search_ListL_L[:self.L]
                search_List=set(search_ListL_L)

            possible_frontier=search_List.difference(Visited)

        search_ListL_L=list(search_List)
        search_ListL_L.sort(key=lambda x: self.get_distance(self.DBGraph.get_vertex(x).value,Query))
        # only maintain k closest points
        search_ListL_L=search_ListL_L[:k]
        search_List=set(search_ListL_L)
        # both are vectors of integers
        return search_List,Visited

    # # Robust pruning
    #candidate set is set of integers
    def Robust_Prune(self,point,candidate_set,alpha):
        # print(candidate_set)
        candidate_set=candidate_set.union(point.neighbors)
        # candidate_set.difference({point.key}) # changed
        candidate_set=candidate_set.difference({point.key}) # changed
        point.neighbors=set()
        # print("candidate_set", candidate_set)
        # while candidate_set not empty
        while candidate_set:
            p_star,_= self.get_min_dist_Key(candidate_set,point.value)
            point.neighbors.add(p_star)
            if(len(point.neighbors)==self.R):
                break
            DummySet=candidate_set.copy()
            for candidatePointKey in candidate_set:
                candidatePoint=self.DBGraph.get_vertex(candidatePointKey)
                # print(alpha * self.get_distance(self.DBGraph.get_vertex(p_star).value,candidatePoint.value), " <= ", self.get_distance(candidatePoint.value,point.value))
                # print(alpha)
                # ">=" condition is reversed as we use cosine similarity (paper uses L2), so higher value means it is closer
                if(alpha * self.get_distance(self.DBGraph.get_vertex(p_star).value,candidatePoint.value) <= self.get_distance(candidatePoint.value,point.value)):
                    # print("HEEEEEEEEEEEEEEEEELPPPPPPPPPPP !!!!!!!!!!!!!!!!!!!!!!")
                    DummySet.remove(candidatePoint.key)
            candidate_set=DummySet


    def Build_Index(self):

        # R = min(R, len(dataset))
        self.iterationOverGraph(1) #alpha=1
        self.iterationOverGraph(self.alpha) #alpha=2



    def iterationOverGraph(self,alpha):
        # print('medoid',medoid)
        randIndex = list(self.DBGraph.get_vertices())
        random.shuffle(randIndex)
        # random permutation + sequential graph update
        for n in randIndex:
            node = self.DBGraph.get_vertex(n)
            # print(n)

            (_,V) = self.Greedy_Search(self.medoid, node.value, 1) #K=1
            
            self.Robust_Prune(node, V, alpha)
            
            neighbors = node.get_neighbors()

            for inbKey in neighbors:

                # CHECK : The backward edge is always added
                # check here in case we shouldn't add it in all cases ? Might be incorrect?
                inb=self.DBGraph.get_vertex(inbKey)
                if len(inb.get_neighbors()) > self.R:
                    # print("inb.get_neighbors()", inb.get_neighbors())
                    U = inb.get_neighbors().union({node.key})
                    # inb.add_neighbor(node.key)
                    self.Robust_Prune(inb, U, alpha)
                else:
                    inb.add_neighbor(node.key)

# Main cell

In [165]:
R=12  # R > log n
k=10
L=10
alpha=2
path='./dataset/DB1m.bin'
step = 100_000
# Cluster files
with open(path, 'rb') as f:
    data = pickle.load(f)
    # print(data.shape)
    for i in range(0, data.shape[0],step):
        temp = data[i:(i+step)]
        # temp[0] = temp[0] - i
        # print(temp.shape)
        # print(i,(i+step))
        pickle.dump(temp, open(f'./dataset/1Mbinaries/{i//step}.bin',"wb"))


In [9]:
R=18  # R > log n
k=10
L=20
alpha=2
path='./dataset/1Mbinaries/'
indexPath = './index/1Mindex/'

offsets=[]
# Loop on clustered files
for idx, filename in enumerate(os.listdir(path)):
    filepath = os.path.join(path, filename)
    
    print(filepath)
    # print(filepath)
    # data = pickle.load(open(filepath,"rb"))
    # print(data)
    # myDB= Indexer(filepath, R, L, alpha, k)
    # myDB.Build_Index()
    # # Write the index on disk
    # pickle.dump(myDB, open(os.path.join(indexPath, str(idx)+".bin"),"wb"))
    # del myDB

./dataset/1Mbinaries/0.bin
./dataset/1Mbinaries/1.bin
./dataset/1Mbinaries/2.bin
./dataset/1Mbinaries/3.bin
./dataset/1Mbinaries/4.bin
./dataset/1Mbinaries/5.bin
./dataset/1Mbinaries/6.bin
./dataset/1Mbinaries/7.bin
./dataset/1Mbinaries/8.bin
./dataset/1Mbinaries/9.bin


In [7]:
# # Write the index on disk
# pickle.dump(myDB, open('./index/DB1k_index.bin',"wb"))

random_vec=np.random.rand(1,70).T
# Load index
# vectors=np.array([myDB.DBGraph.get_vertex(i).value for i in myDB.DBGraph.get_vertices()])
# print(vectors.shape)
vectors = None
with open('./dataset/DB1m.bin', 'rb') as f:
    vectors = pickle.load(f)
    vectors = vectors.loc[:,1:]

actual_result_ids = np.argsort(vectors.dot(random_vec).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(random_vec)), axis= 1).squeeze().tolist()[::-1]

# Recall

In [10]:

def index_to_distance(myDB,id, query):
    vertex = myDB.DBGraph.get_vertex(id)
    query /= np.linalg.norm(query)
    dist = myDB.get_distance(vertex.value, query)
    return dist

totalK = []
for idx, filename in enumerate(os.listdir(indexPath)):
    filepath = os.path.join(indexPath, filename)
    
    myDB = pickle.load(open(filepath,"rb"))
    TopK,_= myDB.Greedy_Search(myDB.medoid,random_vec, k)
    print(type(myDB))
    totalK.extend([(index_to_distance(myDB, k, random_vec), k+(idx* 100_000)) for k in TopK])

    del myDB

k = 5
totalK.sort()

totalK = [k[1] for k in totalK[:k]]

# actual eqn to be tested with
# print('The vector is:')
# print(random_vec)
print('------------------------')
print('len of TopK is:',len(totalK))

# print(TopK.shape)
# for lol in TopK:
    # print(lol)
    # print(myDB.get_distance(myDB.DBGraph.get_vertex(lol).value,random_vec))
    # print('------------------------')


def evaluate_result(results):
    # scores are negative. So getting 0 is the best score.
    scores = []
    
    # case for retireving number not equal to top_k, socre will be the lowest
    if len(set(results)) != k or len(results) != k:
        # scores.append( -1 * len(results) * k)
        score= -1 * len(results) * k
        return score
    score = 0
    for id in results:
        try:
            # print(id)
            ind = actual_result_ids.index(id)
            if ind > k * 3:
                score -= 1
        except:
            # score -= len(results)
            score -= 1
    scores.append(score)

    return sum(scores) / len(scores)

print('------------------------')
print("topK", totalK)
print("Actual", actual_result_ids[:15])
print(evaluate_result(totalK))

AttributeError: Can't get attribute 'VecDB' on <module '__main__'>

# Convert to binary

In [7]:
def csv_to_binary(csv_file_path, binary_file_path):
    # Read the CSV data
    data = pd.read_csv(csv_file_path, header=None)
    # print(data)
    # Write the data to a binary file
    with open(binary_file_path, 'wb') as f:
        pickle.dump(data, f)

csv_to_binary('dataset/DB1k.csv','dataset/DB1k.bin')

# converting to Networkx


In [ ]:
G = nx.DiGraph()
G.add_nodes_from(range(2))
# G = nx.path_graph(2)

attrs = {0: {"vec": [1,2,3,4]}}
nx.set_node_attributes(G, attrs)
G.nodes[0]["vec"]

attrs = {1: {"vec": [2,3,4]}}
nx.set_node_attributes(G, attrs)
print(G.nodes())
print(G.nodes[1]["vec"])

G.add_edge(0,1)
G.add_edge(1,0)

G.edges()



[0, 1]
[2, 3, 4]


OutEdgeView([(0, 1), (1, 0)])

# Clustering

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN, KMeans
from sklearn import metrics
import pandas

def compute_pca(X, n_components=2):
    """
    Input:
        X: of dimension (m,n) where each row corresponds to a word vector
        n_components: Number of components you want to keep.
    Output:
        X_reduced: data transformed in 2 dims/columns + regenerated original data
    """
    X_reduced = None
    ####################### TODO: Use sklearn.decomposition.PCA to implement this function #####################
    X_reduced = PCA(n_components=n_components).fit_transform(X)
    ###########################################################################################################
    return X_reduced

DB_Path="dataset/"
DB=pandas.read_csv('/content/drive/MyDrive/ADBDatasets/Copy of saved_db_1m.csv',index_col=0 ,header=None, dtype = np.float64)

# with open('/content/drive/MyDrive/ADBDatasets/Copy of saved_db_1m.csv', 'r', newline='') as f:
#     DB = f.readlines()
#     DB = [x.strip() for x in DB]
#     for i,x in enumerate(DB):
#             x=x.split(',')
#             x=np.array(x[1:],dtype=np.float32)
#             DB[i]=x   
    #  print(len(DB))
    # print(DB)


result = compute_pca(DB, 2)
# plt.scatter(result[:, 0], result[:, 1])
# plt.show()
# print(result.shape)
# result=result.reshape(1,-1)
length = np.sqrt((result**2).sum(axis=1))[:,None]
result = result / length
# db = DBSCAN(eps=epsilon, min_samples=10,metric="cosine").fit(result)
# db = KMeans(n_clusters=10).fit(result)
# good eps 0.02 and min_samples=10 for DBSCAN 1k
epsilon=0.005
db = DBSCAN(eps=epsilon, min_samples=10).fit(result)
# print(db.labels_)
print('ulabels',np.unique(db.labels_))
print('len of ulabels',len(np.unique(db.labels_)))
plt.scatter(result[:, 0], result[:, 1], c=db.labels_)
plt.show()